In [32]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import json
import time

url = "https://www.data.go.kr/index.do"
chrome_options = Options()
chrome_options.add_experimental_option("detach", True)

driver = webdriver.Chrome(options=chrome_options)
driver.get(url)
time.sleep(1)

searchValue = input("어떤 공공데이터를 찾으시나요?")
searchPage = input("수집하실 페이지 범위를 입력하세요(예: 1~4): ")
startPage = searchPage.split("~")[0]
endPage = searchPage.split("~")[1]

searchInput = driver.find_element(by=By.CSS_SELECTOR, value='input[class="input-text"]')

searchInput.send_keys(searchValue)
searchInput.send_keys(Keys.RETURN)
time.sleep(5)
driver.find_element(by=By.CSS_SELECTOR, value='button[title="오픈 API 더보기"]').click()
time.sleep(5)

data_list = []

for i in range(int(startPage), int(endPage) + 1):
    print()
    driver.find_element(by=By.XPATH, value=f'//*[@id="apiDataList"]/nav/a[{i + 1}]').click()
    time.sleep(5)  
    contents = driver.find_elements(by=By.CSS_SELECTOR, value='div[class="result-list"] ul li')

    for content in contents:
        title = content.find_element(by=By.CSS_SELECTOR, value='dt a span[class="title"]').text
        
        info_data = content.find_elements(by=By.CSS_SELECTOR, value='div[class="info-data"] p span[class="data"]')
        info_list = []
        for info in info_data:
            info_list.append(info.text)

        keywordSelecter = content.find_elements(by=By.CSS_SELECTOR, value='div[class="info-data"] p')
        findKeyword = []
        for keyword in keywordSelecter:
            findKeyword.append(keyword.text)

        data = {
            'title': title,
            'provider_organizationro': info_list[0],
            'update_date': info_list[1],
            'hits': int(info_list[2]),
            'request_count': int(info_list[3]),
            'keyword': findKeyword[4].split(" ")[1]
        }

        data_list.append(data)

with open("Unit5.json", "w", encoding="utf-8") as f:
    json.dump(data_list, f, indent=4, ensure_ascii=False)